In [90]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from pattern import web
import scipy.stats as stats
from scipy.stats import binom
from __future__ import division
import re

from datetime import datetime

#nice defaults for matplotlib
from matplotlib import rcParams

dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843),
                (0.4, 0.4, 0.4)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = '#eeeeee'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'none'

thep= 0.5534

In [85]:
currentPage = 0
table = []

while True:
    URL = "https://www.themoviedb.org/tv/on-the-air?page=%d" % (int(currentPage) + 1)
    html = requests.get(URL).text
    dom = web.Element(html)
    p = dom.by_class('left')[0].content
    currentPage, pagesCount = re.findall("[-+]?\d+[\.]?\d*", p)
    if currentPage == pagesCount:
        break
    rows = dom.by_class('info')
    
    
    for row in rows:
        heading = row('h4')[0]
        title = heading('a')[0].content
        year = heading('span')[0].content.replace('(', '').replace(')','')
        favourite = row.by_class('favourite')[0].content
        watchlist = row.by_class('watchlist')[0].content
        r = row.by_class('rating')[0].content
        cast = row('p')[0]
        c = []
        c1 = cast('a')
        for i in c1:
            c.append(web.plaintext(i.content))
        l = ", ".join(c)
        rating, votes = re.findall("[-+]?\d+[\.]?\d*", r)
        #print "Title: %s, year: %s" %(title, year)
        #print "favourite: ", favourite
        #print web.plaintext(row.content)
        table.append([title, year, favourite, watchlist, rating, votes, l])
    
df = pd.DataFrame(table, columns=['Title', 'Year', 'Favourites', 'Watchlist', 'Rating', 'Votes', 'Cast'])
df

,Title,Year,Favourites,Watchlist,Rating,Votes,Cast
0,Game of Thrones,2011,310,259,9.1,251,"David Benioff, D. B. Weiss, Peter Dinklage, Ni..."
1,American Dad!,2005,21,20,7.7,24,"Seth MacFarlane, Wendy Schaal, Rachael MacFarlane"
2,Outlander,2014,30,42,8.1,22,"Caitriona Balfe, Sam Heughan, Lotte Verbeek"
3,Community,2009,14,13,7.7,40,"Joel McHale, Gillian Jacobs, Danny Pudi"
4,Girl Meets World,2014,1,2,7.5,1,"Ben Savage, Danielle Fishel, Rowan Blanchard"
5,Finding Carter,2014,3,3,5.8,6,"Kathryn Prescott, Cynthia Watros, Alexis Denisof"
6,Wayward Pines,2015,7,13,9.3,2,"Matt Dillon, Carla Gugino, Melissa Leo"
7,Aquarius,2015,0,2,0.0,0,"David Duchovny, Grey Damon, Gethin Anthony"
8,500 Questions,2015,0,0,0.0,0,Richard Quest
9,Bones,2005,29,32,6.7,26,"Emily Deschanel, David Boreanaz, Michaela Conlin"


In [112]:
def getNews(datetimeUntil):    
    currentPage = 1
    table = []
    while True:
        URL = "http://www.thehollywoodgossip.com/page-%d.html" % currentPage
        html = requests.get(URL).text
        dom = web.Element(html)
        rows = dom.by_class('col-sm-6')
        
        for row in rows:
            heading = row('h2')[0]
            publishingDate = row('time')[0]
            dictPublishingDate = publishingDate.attrs
            d = dictPublishingDate['datetime']

            d_mod = d[:d.rfind('-')]
            datetime_obj = datetime.strptime(d_mod, '%Y-%m-%dT%H:%M:%S')
            if datetime_obj < datetimeUntil:
                return table
            table.append([heading.content, datetime_obj])

        currentPage += 1
        
        
datetimeUntil = datetime(2015,5,25)
table = getNews(datetimeUntil)
        
df = pd.DataFrame(table, columns=['Title', 'Date'])
df

,Title,Date
0,21 Bisexual Celebrities: You Won't Believe #7!,2015-05-28 07:42:52
1,"Bobby Flay Banged January Jones, Many Others, ...",2015-05-28 09:18:57
2,Dad Runs Over Son's Video Games with a Lawn Mower,2015-05-28 09:08:32
3,Brokeback Bachelorette Promo Teases Same-Sex S...,2015-05-28 08:56:02
4,Michelle and Jim Bob Duggar: Focused on Family...,2015-05-28 08:42:11
5,Game of Thrones Episode Teaser: Breaking Bread...,2015-05-28 08:40:30
6,Amy Schumer as The Bachelorette: Is She In?!?,2015-05-28 07:32:43
7,"Courtney Stodden, Momager Split Over &quot;Cre...",2015-05-28 07:10:32
8,Dwayne Johnson Throws Selfie Shade at Kim Kard...,2015-05-28 07:09:42
9,Demi Lovato Thanks Phil McIntyre for LITERALLY...,2015-05-28 06:56:13
